In [ ]:
!pip install transformers
!pip install fasttext

In [61]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf 
from transformers import BertTokenizer

from copy import deepcopy

from typing import Union, List

from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer
import fasttext
from copy import deepcopy
import numpy as np
import csv
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
dataset = pd.read_csv (r'dataset.csv',names=['description'])
classes = []
labels=[]
catagories = ["__label__bug","__label__enhancement","__label__question"]


for line in range(0,len(dataset)):
    class_name = dataset.iloc[line].to_string(header=False, index=False).split(" ")[0]
    if class_name  in catagories:
      if class_name == "__label__bug":
        classes.append("__label__bug")
        labels.append(0)
      elif class_name == "__label__enhancement":
        classes.append("__label__enhancement")
        labels.append(1)
      else:
         classes.append("__label__question")
         labels.append(2)
      dataset.iloc[line] = dataset.iloc[line].str.replace(class_name,'')
    else:
      labels.append(pd.NaT)
      classes.append(pd.NaT)
      dataset.iloc[line] = pd.NaT

dataset['labels'] = labels
dataset['class'] = classes

dataset.dropna(axis = 0, how = 'all', inplace = True)
dataset = dataset.reset_index()
ds = deepcopy(dataset)




# **Fasttext model**

In [ ]:
target = dataset['class']

x=dataset['description']

stopword=stopwords.words('english')
ps=PorterStemmer()
x=dataset['description']

for i in range(len(x)):
  word=re.sub('[^a-zA-Z]',' ',x[i])
  word=word.lower()
  word=word_tokenize(word)
  word=[ps.stem(words) for words in word if words not in stopword]
  x[i]=' '.join(word)


cv=CountVectorizer(max_features=3000)
x_vector=cv.fit_transform(x).toarray()

frames = [target, x]
dataset = pd.concat(frames, axis=1)
train_size = int(len(dataset)* 0.8)
ds1 = dataset[:train_size]
ds2 = dataset[train_size+1:]


ds1[['class', 'description']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")


ds2[['class', 'description']].to_csv('test.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")



In [7]:
model = fasttext.train_supervised('train.txt', wordNgrams = 0, loss='softmax',minCount= 14, epoch = 500)

In [ ]:
model.test(test.txt)

In [10]:
model.predict(ds.iloc[1,1])

(('__label__enhancement',), array([0.99720573]))

# **Bert Model**

In [63]:
# data_bug = ds[ds['labels']  == 0] 
# data_enhancement = ds[ds['labels']  == 1] 
# data_question = ds[ds['labels']  == 2] 

# data_bug = data_bug[:4000]
# data_enhancement = data_enhancement[:4000]
# data_question = data_question[:4000]

# dataset = pd.concat([data_bug, data_enhancement,data_question], axis=0)

# dataset = dataset.sample(frac=1).reset_index()
# len(dataset)
# dataset.head()

# removing leading and trailing space 

ds1 = deepcopy(ds)
dataset = ds.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [64]:
dataset['labels'].value_counts()

0    16355
1    14228
2     3458
Name: labels, dtype: int64

In [65]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [66]:
X_input_ids = np.zeros((len(dataset),256))
X_attension_masks = np.zeros((len(dataset),256))

In [67]:
X_input_ids.shape

(34041, 256)

In [68]:
def training_data_prep(df, ids, masks, tokenizer):
  for i, text in tqdm(enumerate(df['description'])):
    tokenized_text = tokenizer.encode_plus(
        text,
        max_length = 256,
        truncation = True,
        padding = 'max_length',
        add_special_tokens = True,
        return_tensors = 'tf'
      )
    ids[i,:] = tokenized_text.input_ids
    masks[i,:] = tokenized_text.attention_mask
  return ids,masks

In [69]:
X_input_ids,X_attension_masks = training_data_prep(dataset,X_input_ids,X_attension_masks,tokenizer)

0it [00:00, ?it/s]

In [70]:
X_input_ids

array([[  101.,   169., 19722., ...,     0.,     0.,     0.],
       [  101.,   156., 18548., ...,     0.,     0.,     0.],
       [  101.,  3799.,  1403., ...,  2137., 18922.,   102.],
       ...,
       [  101.,   193., 22433., ...,     0.,     0.,     0.],
       [  101.,  1960.,   189., ...,     0.,     0.,     0.],
       [  101.,  1960.,   189., ...,     0.,     0.,     0.]])

In [71]:
labels = np.zeros((len(dataset),3))

In [72]:
labels[np.arange(len(dataset)), dataset['labels'].values] = 1

In [73]:
ds = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attension_masks,labels ))

In [74]:
def mapFunction(input_ids, attn_masks, labels):
  return {
      'input_ids':input_ids,
      'attention_mask': attn_masks,
  },labels

In [75]:
ds = ds.map(mapFunction)

In [76]:
ds.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [77]:
ds = ds.shuffle(10000).batch(16, drop_remainder= True)

In [78]:
split = 0.8
train_size = int((len(dataset)//16) * split)


In [79]:
train_ds = ds.take(train_size)
val_size = int((len(train_ds)) * split)
train_dataset = train_ds.take(val_size)
val_dataset = train_ds.skip(val_size)
test_dataset = ds.skip(train_size)

print(len(train_dataset), len(val_dataset), len(test_dataset))

1360 341 426


In [80]:
from transformers import TFBertModel

In [81]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [82]:
input_ids = tf.keras.layers.Input(shape=(256,),name='input_ids',dtype='int32')
attention_mask = tf.keras.layers.Input(shape=(256,),name='attention_mask',dtype='int32')


berd_embds = bert_model.bert(input_ids, attention_mask=attention_mask)[1]
intermediate_layer = tf.keras.layers.Dense(512, activation = 'relu', name = 'intermediate_layer')(berd_embds)
output_layer = tf.keras.layers.Dense(3, activation = 'softmax', name = 'output_layer')(intermediate_layer)

model = tf.keras.Model(inputs = [input_ids, attention_mask], outputs = output_layer)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [83]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, decay = 1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [84]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [85]:
model.compile(optimizer = optimizer, loss = loss_func,  metrics=[acc,f1_m,precision_m, recall_m])

In [86]:
hist = model.fit(
    train_dataset, validation_data = val_dataset, epochs = 4
)

Epoch 1/4
1360/1360 [==============================] - 1273s 902ms/step - loss: 0.5898 - accuracy: 0.7608 - f1_m: 0.7485 - precision_m: 0.7816 - recall_m: 0.7244 - val_loss: 0.4913 - val_accuracy: 0.8167 - val_f1_m: 0.8110 - val_precision_m: 0.8294 - val_recall_m: 0.7944
Epoch 2/4
1360/1360 [==============================] - 1227s 902ms/step - loss: 0.4848 - accuracy: 0.8141 - f1_m: 0.8124 - precision_m: 0.8296 - recall_m: 0.7970 - val_loss: 0.4170 - val_accuracy: 0.8438 - val_f1_m: 0.8429 - val_precision_m: 0.8540 - val_recall_m: 0.8328
Epoch 3/4
1360/1360 [==============================] - 1226s 902ms/step - loss: 0.4096 - accuracy: 0.8465 - f1_m: 0.8460 - precision_m: 0.8585 - recall_m: 0.8347 - val_loss: 0.3997 - val_accuracy: 0.8552 - val_f1_m: 0.8568 - val_precision_m: 0.8651 - val_recall_m: 0.8492
Epoch 4/4
1360/1360 [==============================] - 1226s 902ms/step - loss: 0.3441 - accuracy: 0.8742 - f1_m: 0.8733 - precision_m: 0.8830 - recall_m: 0.8644 - val_loss: 0.3443 - v

In [96]:
model.predict(test_dataset)

array([[0.3915886 , 0.51777864, 0.0906328 ],
       [0.932819  , 0.01084513, 0.05633579],
       [0.0840931 , 0.90801936, 0.0078876 ],
       ...,
       [0.98971295, 0.00696268, 0.00332439],
       [0.27757618, 0.7150035 , 0.00742036],
       [0.0403362 , 0.95542765, 0.00423614]], dtype=float32)

In [97]:
test_dataset.take

TypeError: ignored

In [87]:
model.save('issue_labelling_model')

INFO:tensorflow:Assets written to: issue_labelling_model/assets


INFO:tensorflow:Assets written to: issue_labelling_model/assets


In [ ]:
# load_model = tf.keras.models.load_model('issue_labelling_model')

In [89]:
# from google.colab import drive
# drive.mount('/content/drive')

In [90]:
import shutil
shutil.make_archive('issue_labelling_model', 'zip', 'issue_labelling_model')

'/content/issue_labelling_model.zip'

In [91]:
! unzip -q issue_labelling_model.zip